# PDE Code Documentation 

This section of the documentation will discuss the Python implementation of Finite Element Method (FEM) solvers.

## Theory

**TODO (Victor). Much will be copied from slides, with better written text.

### Finite Element Method Theory

### Elliptic Equation: Obtaining Variational Form   #Leo: I'd write elliptic model problem instead of elliptic equation, as just a suggestion

### Parabolic Equation: Obtaining Variational Form


## Setting up PDE Code environment

Todo (Victor) .Briefly mention Docker setup of FEniCS, and state where the dependencies.txt file is.

## Test Cases for Optimization  #Leo: I'm not sure if we should call these test cases, because this may also cause confusion with the test cases on the PDEs we have in pde_test

This section contains brief instructions on how to generate the test cases used in the presentation. Afterwards, how each script works will be broken down in detail.

### Commands to Generate Test Cases

The two elliptic (time-independent) test cases were:
 - Elliptic Equation, Single Gaussian Point Source RHS: #Leo: it may be beneficial to avoid "point source" (zb only "source"), this can be confused with a measure
     - Generated by command `python elliptic_ex.py -y elliptic_params.yaml` 
     - Resulting pickle file saved to `elliptic_params.pkl`
 - Elliptic Equation, Multiple Gaussian Point Sources RHS:
     - Generated by command `python elliptic_ex.py -y elliptic_multi_gaussian.yaml` 
     - Resulting pickle file saved to `elliptic_multi_gaussian.pkl`
     
The two parabolic (time-dependent) test cases were:
 - Parabolic Equation, Slow-moving Gaussian Point Source RHS:
     - Generated by command `python parabolic_ex.py -y par_moving_bump_slow.yaml` 
     - Resulting pickle file saved to `par_moving_bump_slow.pkl`
 - Parabolic Equation, Fast-moving Gaussian Point Source RHS:
     - Generated by command `python parabolic_ex.py -y par_moving_bump_fast.yaml` 
     - Resulting pickle file saved to `par_moving_bump_fast.pkl` 

 

## PDE Solver Files

This section goes into more detail about how the PDE solver files generate solutions. Each section shows key functions used to generate the source code and gives a motivating example.

### Elliptical PDE Solver   #Leo: elliptic? Not sure

We document the PDE solver by first showing some key functions used in it and then displaying a contour plot of its solution. Finally, we give a quick test example demonstrating that the solution behaves according to its order of convergence.

#### Example Parameter File - Single Gaussian Point Source
Parameters describing how to run the elliptical solver are stored in the `EllipticRunParams` struct in `interface.py`. These parameters are stored in text form using the .YAML file format, and read in using the `PyYAML` library **insert citation**. 

The indentation in the following example parameter file corresponds to the `EllipticRunParams` struct that is generated. For instance, the struct determined by the yaml_file has sub-elements with names `var_form`, `rect_mesh`, and `io`.

As another example, the element P0 is accessed by `in_params.rect_mesh.P0`, given that the EllipticRunParams object is called `in_params`.

In [ ]:
# %load PDEFEMCode/Pickles/elliptic_params.yaml
# This is a YAML file which contains sample parameters to run elliptic_ex.py
# Note: Please note that the indentation is important to the processing of this file.
# Lines starting with '# 'are comments and are ignored, as in Python.
# Variational form is elliptic with a Gaussian Point source RHS.

# Parameters related to PDE variational form.
var_form:
  # Parameters which define the variational form type.
  # Elliptic_LHS and Elliptic_RHS forms used together solve
  # \int \nabla u \cdot \nabla v + uv dx = \int f*v dx
  # (The weak formulation of -\nabla^2 u + u = f).
  #Note: Homogeneous Neumann Boundary conditions are assumed on the unit square.

  LHS_form_str:    'elliptic_LHS'
    # function defined in fenics_utils.py.
  RHS_form_str:    'elliptic_RHS'
    # function defined in fenics_utils.py

  #Parameters related to the RHS
  rhs_expression_str: '' # Instead of a function which constructs a string, one can choose to
                         # use a valid FEniCS expression. See parabolic_params.yaml for an example.
  rhs_expression_fn_str: 'gaussian_expression_2D' # Gaussian RHS function f is of form exp(-norm(x-gamma)^2/r).
                                              # The function which generates the correct FEniCS string is called
                                              # gaussian_expression_2D and is defined in fenics_utils.py.
  rhs_exp_params: #keyword arguments to the function in rhs_expression_fn_str. If no function is provided, please use {}, as in parabolic_params.yaml.
    gamma: [ .7,.5 ] # coordinates of Gaussian point source
    u_max: 1        # scaling constant which defines the
                    #   maximum of the function used in the RHS.
    r: .05          # radius parameter

    #Note: Homogeneous Neumann Boundary conditions are assumed on the unit square.

# Parameters determining the mesh.
# As is, it is a square 0.01 step size mesh on the unit square
rect_mesh:
  nx: 100 # number of divisions of the unit interval in x, included in the mesh.
  # There are nx + 1 distinct x coordinates in the mesh.
  ny: 100 # same as above, for y.
  P0: [0,0] # Coordinates of the bottom left of the rectangular mesh.
  P1: [1,1] # Coordinates of the top right of the rectangular mesh

# IO Properties
# Parameters which determine where the file is saved.
io:
  out_file_prefix: 'elliptic_params' #file name without the default .pkl extension.
  out_folder: 'PDEFEMCode/Pickles' #Path to file. Leave blank to save in the current directory.

#### Key functionality of file: Source code 

To show the full functionality of the source code, we display the variational forms mentioned in the given parameter file (as well as the RHS expression generator) and then display the code which solves a variational equation.

In [ ]:
# %load -s elliptic_LHS,elliptic_RHS,gaussian_expression_2D PDEFEMCode/fenics_utils.py
def elliptic_LHS(u_trial, v_test, **kwargs):
    '''
    returns the LHS of the elliptic problem provided in the project handout:
    -\Delta u + u = f  => \int (grad(u) dot grad(v)  + u*x) dx = \int f*v dx
    :param u_trial: The trial function space defined on the FENICS Fn space.
        (You obtain this by calling u_trial = fc.TrialFunction(V))
    :param v_test: The test function space defined on the FENICS Fn space.
        (You obtain this by calling u_trial = fc.TestFunction(V))
    :return: an integral form of the equation, ready to be used in solve_pde.
    '''
    return (fc.dot(fc.grad(u_trial), fc.grad(v_test)) + u_trial * v_test) * fc.dx

def elliptic_RHS(v_test, RHS_fn, **kwargs):
    '''
    returns the RHS of the elliptic problem provided in the project handout:
    \Delta u + u = f  => \int (grad(u) dot grad(v)  + u*x) dx = \int f*v dx
    :param v_test: The test function space defined on the FENICS Fn space.
        (You obtain this by calling u_test = fc.TestFunction(V))
    :param RHS_fn: a FENICS function evaluated on the function space.
    :return: an integral form of the equation, ready to be used in solve_pde.
    '''
    return RHS_fn * v_test * fc.dx

def gaussian_expression_2D(gamma,u_max,r):
    '''
    Returns a fenics string which represents a Gaussian point source in 2D.
    :param gamma: shape (2,) the center of the gaussian point source
    :param u_max: Height scaling parameter.
    :param r: radius parameter.
    :return: the string representation
    '''
    return '{} * exp(-(pow(x[0] - {},2) + pow(x[1] - {},2)) /pow({},2))'.format(u_max,gamma[0],gamma[1], r)


See the previous section Example Parameter file for a brief description of in_params.

In [ ]:
# %load -s solve_single_variation_form PDEFEMCode/elliptic_ex.py


#### Example: Using Custom Function Interpolators

In [4]:
# # Code snippet to build intuition on the objects.
# ---TODO (Victor): Generate plot.--------
# import matplotlib.pyplot as plt
# coords = np.stack(np.meshgrid(np.linspace(0,1,200),np.linspace(0,1,200)),axis = -1)
# coords_rs = coords.reshape(-1,2)
# f_eval = f(coords_rs)
# plt.imshow(f_eval.reshape(200,200))
#
# grad_f_eval = grad_f(coords) #grad_f(coords_rs) works fine too
# plt.imshow(grad_f_eval[:,:,0])


### Example: Order of Convergence Test with Plot

**TODO (Victor)**

### Parabolic PDE Solver

We document the parabolic PDE solver in an analogous way. First we show the parameters which determine how the parabolic server runs. Then 

#### Example Parameter File - Slow-Moving Gaussian Point Source
Parameters describing how to run the elliptical solver are stored in the `ParabolicRunParams` struct in `interface.py`. As in the elliptical case, these parameters are stored in text form using the .YAML file format.

Just as before, the element P0 is accessed by `in_params.rect_mesh.P0`, given that the ParabolicRunParams object is called `in_params`. #Leo: I wouldn't use element to indicate a point, in this context

In [ ]:
# %load PDEFEMCode/Pickles/par_moving_bump_slow.yaml
# This is a YAML file which contains sample parameters to run parabolic_ex.py
# Note: Please note that the indentation is important to the processing of this file.
# Lines starting with '# 'are comments and are ignored, as in Python.

# Parameters related to PDE variational form.
var_form:
  # todo: insert comment about parabolic LHS and RHS, then copy paste it to other parameter files.
  LHS_form_str:    'heat_eq_LHS'
    #Function defined in fenics_utils.py
  RHS_form_str:    'heat_eq_RHS'
    #function defined in fenics_utils.py
  rhs_expression_str: '100*exp(-(pow(x[0]-0.5-0.04*t*cos(t),2)+pow(x[1]-0.5-0.04*t*sin(t),2))/pow(0.05,2))'
    # The FEniCS expression which when evaluated, will be used as f in the right hand side of the variational form.
    # This is a function of x[0],x[1] and t.
  rhs_expression_fn_str: '' # Alternately, one may choose a function which
                            # generates a valid FEniCS string. See elliptic_params for an example
  rhs_exp_params: {} #any extra keyword arguments to the function in rhs_expression_fn_str. If empty, use {}.
  #Note: Homogeneous Von Neumann Boundary conditions are assumed on the unit square.


# Parameters determining the mesh.
# As is, it is a square 0.05 step size mesh on the unit square
rect_mesh:
  nx: 100 # number of divisions of the unit interval in x, included in the mesh.
  # There are nx + 1 distinct x coordinates in the mesh.
  ny: 100 # same as above, for y.
  P0: [0,0] # Coordinates of the bottom left of the rectangular mesh.
  P1: [1,1] # Coordinates of the top right of the rectangular mesh

# Parameters related to the time discretization of the parabolic problem.
time_disc:
  T_fin: 10 #Ending time
  Nt: 100 # If -1, the default is nx*ny distinct timesteps.

# IO Properties
io:
  out_file_prefix: 'par_moving_bump_slow' #file name without the default .pkl extension.
  out_folder: 'Pickles'#Path to file. Leave blank to save in the current directory.



#### Key functionality of file: Source code 

Just as before, we show the full functionality of the source code, we display the variational forms mentioned in the given parameter file (as well as the RHS expression generator) and then display the code which solves a variational equation. 
Note: 
 - The variational form of the heat equation was generated using the Implicit Euler method with time step `dt`, therefore requires `u_previous`, the solution at the previous timestep.
 - There is no function which generates a fenics expression string. This is because the full string is hard-coded in `var_form.rhs_expression_str` in the parameter file above.
 - A finite element solver is evaluated `time_disc.Nt` times after the initial time, where $u(0) = 0$.  #Leo: I would write instead "is run" instead of evaluated, and $u(0,x)=0$ for every x

In [ ]:
# %load -s heat_eq_LHS,heat_eq_RHS PDEFEMCode/fenics_utils.py
def heat_eq_LHS(u_trial, v_test, dt=1, alpha=1):
    '''
    returns the LHS a(u_trial, v) of the parabolic problem provided in the project handout:
    D_t u  - \alpha \Delta u = f, u(0)=0, homogeneous Neumann BC and initial conditions
    as discretized by means of the implicit Euler's method =>
    a(u_trial, v) = \int u_trial * v dx + \int dt * alpha * dot(grad(u_trial), grad(v)) dx
    L(v) = (u_previous + dt * f) * v * dx
    u_trial is the solution at current time, u_previous is at last time, and not needed in this routine (see below)

    If alpha=dt=1 returns the LHS of the elliptic problem provided in the project handout:
    -\Delta u + u = f  => \int (grad(u) dot grad(v)  + u*x) dx = \int f*v dx

    :param u_trial: The trial function space defined on the FENICS Fn space.
        (You obtain this by calling u_trial = fc.TrialFunction(V))
        Note: it is the solution of the current time step
    :param v_test: The test function space defined on the FENICS Fn space.
        (You obtain this by calling v_trial = fc.TestFunction(V))
    :param dt: time discretization mesh size. The default is 1, so that a stationary PDE can also be approximated
    :param alpha: a constant, default 1

    :return: an integral form of the equation, ready to be used in solve_pde.
    '''

    return (dt * alpha * fc.dot(fc.grad(u_trial), fc.grad(v_test)) + u_trial * v_test) * fc.dx

def heat_eq_RHS(v_test, RHS_fn, dt=1, u_previous=0):
    '''
    returns the RHS L(v) = (u_previous + dt * f) * v * dx of the parabolic problem provided in the project handout:
    D_t u  - \alpha \Delta u = f, u(0)=0, homogeneous Neumann BC and initial conditions
    as discretized by means of the implicit Euler's method =>
    a (u_trial, v) = \int u_trial * v dx + \int dt * alpha * dot(grad(u_trial), grad(v)) dx
    L (v) = (u_previous + dt * f) * v * dx
    u_trial is the solution at current time, not needed here, u_previous is at last time (see below)

    If dt=1, u_previous=0, returns the RHSof the elliptic problem provided in the project handout:
    \Delta u + u = f  => \int (grad(u) dot grad(v)  + u*x) dx = \int f*v dx

    :param u_previous: the FEM solution at the last time step (whereas u_triaL is at current time step). Default: 0
        (You obtain this by calling u_trial = fc.TestFunction(V))
    :param dt: time discretization mesh size. Default: 1
    :param v_test: The test function space defined on the FENICS Fn space.
        (You obtain this by calling v_trial = fc.TestFunction(V))
    :param RHS_fn: a FENICS function evaluated on the function space.
        (obtained by calling fc.
    :return: an integral form of the equation, ready to be used in solve_pde.
    '''

    return (dt * RHS_fn + u_previous) * v_test * fc.dx


In [ ]:
# %load -s solve_parabolic_problem PDEFEMCode/parabolic_ex.py
def solve_parabolic_problem(in_params):
    '''
    Solves a single PDE with variational forms given in the parameter struct.
    The mesh structure is assumed to be a rectangular Friedrics-Keller triangulation,
        the same as the interpolator structure assumes.
    Input/Output:
    :param in_params: type EllipticRunParams
        A Dataclass struct containing all parameters necessary to run a single solution.
    :return: tuple:
        param_save: a dictionary containing:
            f: function interpolator class FenicsRectangleLinearInterpolator
            grad_f: gradient evaluator class    FenicsRectangleVecInterpolator
            param: the parameter struct used to run this script.
    '''

    # For convenience, separate master struct into separate variables.
    var_form_p = in_params.var_form
    var_form_fn_handles = pde_utils.VarFormFnHandles(var_form_p)
    mesh_p = in_params.rect_mesh
    time_disc_p = in_params.time_disc
    io_p = in_params.io

    # Space discretization
    mesh, fn_space = pde_utils.setup_rectangular_function_space(mesh_p)

    # Time discretization
    dt, times = pde_utils.setup_time_discretization(time_disc_p)

    # Setup variational formulation, tying the LHS form with the trial function
    # and the RHS form with the test functions and the RHS function.
    u_initial = fc.interpolate(fc.Constant(0), fn_space)  # the solution is zero according to our problem formulation
    u_trial = fc.TrialFunction(fn_space)
    v_test = fc.TestFunction(fn_space)

    if not var_form_p.rhs_expression_str:
        var_form_p.rhs_expression_str = var_form_fn_handles.rhs_expression(**var_form_p.rhs_exp_params)
    RHS_fn = fc.Expression(
        var_form_p.rhs_expression_str, degree=2, t=0)

    u_previous = u_initial
    LHS_int, RHS_int = pde_utils.variational_formulation(
        u_trial, v_test,
        var_form_fn_handles.LHS,
        var_form_fn_handles.RHS, RHS_fn,
        {'dt': dt},
        {'dt': dt, 'u_previous': u_previous}
    )

    t = times[0]  # initial time
    fenics_list = []  # list of Fenics functions to be passed to the interpolators

    # Start main solving block.
    fc.set_log_active(False)  # disable messages of Fenics
    for n in range(time_disc_p.Nt):
        # Time update
        t = times[n + 1]
        RHS_fn.t = t  # NB. This change is also reflected inside LHS_int, RHS_int

        # Solution at this time, and update previous solution
        u_current = pde_utils.solve_vp(fn_space, LHS_int, RHS_int)
        u_previous.assign(u_current)
        fenics_list.append(u_current)

        # Notify that something has happened
        print("Solving PDE, " + str(np.floor(100 * t/time_disc_p.T_fin)) + " % done.")

    # Prepending initial condition
    fenics_list.insert(0, fc.interpolate(fc.Constant(0), fn_space))

    # Obtaining the interpolator for u
    u = pde_interface.FenicsRectangleLinearInterpolator(mesh_p, fenics_list, T_fin=time_disc_p.T_fin, Nt=time_disc_p.Nt, time_dependent=True, verbose=True, time_as_indices=True)
    grad_u_list = [pde_utils.fenics_grad(mesh,u_fenics) for u_fenics in fenics_list]
    grad_u = pde_interface.FenicsRectangleVecInterpolator(mesh_p, grad_u_list, T_fin=time_disc_p.T_fin, Nt=time_disc_p.Nt, time_dependent=True, time_as_indices=True)

    # Organize and save final results.
    param_save = {'f':u,'grad_f':grad_u,'params':in_params}
    return param_save,fenics_list,grad_u_list


## Modularity of Code

Mention Independence of Interpolators from FEniCS. Mention that the method for doing this is inside of the main doc section.

## Unit Tests

Mention Python's Unittests library, simple numerical tests that ensure the code can be easier checked.